In [1]:
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/home/micah/Desktop/data/data.txt', sep=",", header=None)
df.columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'target']


pd.set_option('display.max_columns', 999)

print df.head(n=3)
print '\n'
print df.info()
print '\n'


print (df[7].isnull().sum()/len(df)) * 100

         1  2  3  4  5  6   7  8  9  10  target
0  1000025  5  1  1  1  2   1  3  1   1       2
1  1002945  5  4  4  5  7  10  3  2   1       2
2  1015425  3  1  1  1  2   2  3  1   1       2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
1         699 non-null int64
2         699 non-null int64
3         699 non-null int64
4         699 non-null int64
5         699 non-null int64
6         699 non-null int64
7         699 non-null object
8         699 non-null int64
9         699 non-null int64
10        699 non-null int64
target    699 non-null int64
dtypes: int64(10), object(1)
memory usage: 60.1+ KB
None


0


In [3]:


#df[7] = df[7].map(lambda x: x.strip('?') )
print df[7].groupby(7).count()

df[7]= df[7].replace('?', df[7].mode()[0]) 
#df[7]= df[7].replace('?', '') 

#df = df.dropna() 
print df[7].unique() 
df[7] = pd.to_numeric(df[7])
'''
df= df.fillna(df.mean())
'''

TypeError: 'str' object is not callable

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

one = scaler.fit_transform(df[1].values.reshape(-1, 1))

#df[1] = scaler.fit_transform(df[1].values.reshape(-1, 1))
df = df.drop([1], axis=1)

df[11] = pd.DataFrame(one)
df[11]= df[11] + 10
print 'max ', df.max()
print 'min ', df.min()
#rint df.head()

print df[7].nunique()
print df[7].unique()

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df['target'], test_size=0.2, random_state=42)


In [ ]:

p_grid = {"C": [1, 10, 100],"gamma": [.01, .1]}

NUM_TRIALS = 20

non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# We will use a Support Vector Classifier with "rbf" kernel
svm = SVC(kernel="rbf")

# Loop for each trial
for i in range(NUM_TRIALS):

    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv,iid=False)
    clf.fit(X_train, y_train)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=X_train.values, y=y_train.values, cv=outer_cv)
    nested_scores[i] = nested_score.mean()
print non_nested_scores.mean()    
print  nested_scores.mean()
